In [1]:
import pandas as pd
import keras
import numpy as np

# Import CIFAR 10 dataset
from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator

# Import Necessary CNN Building Blocks
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU

from keras import backend as K
from keras import applications
from keras import callbacks
from keras import optimizers
from keras.utils import np_utils
import random

import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Requirements:
# !pip install keras
# !pip install tensorflow
# !pip install keras
# !pip install tensorflow_addons

# TODO: data augmentation validation

## <span style='font-family:Georgia'> Data preparation

In [3]:
# Load Dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

BATCH_SIZE = 32
INPUT_SHAPE = X_train.shape[1:]
NUM_CLASSES = 10

print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [4]:
# Normalize input data
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Encode target variable
y_train_ohe = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test_ohe = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

## <span style='font-family:Georgia'> Deep Learning pretrained models fitting

### <span style='font-family:Georgia'> VGG16

In [23]:
EPOCHS=3  # 50
BATCH_SIZE = 32
LR=1e-3
MOMENTUM=0.9
INPUT_SHAPE = X_train.shape[1:]

In [24]:
vgg_base = tf.keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',input_shape=INPUT_SHAPE)

vgg = Sequential()
vgg.add(vgg_base)
vgg.add(Flatten()) 
vgg.add(Dense(NUM_CLASSES, activation=('softmax')))
vgg.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,719,818
Trainable params: 14,719,818
Non-trainable params: 0
_________________________________________________________________


In [25]:
vgg.compile(loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM),
            metrics=['accuracy'])

In [26]:
vgg_history = (
    vgg.fit(X_train,
            y_train_ohe, 
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            steps_per_epoch=X_train.shape[0]//BATCH_SIZE,
            validation_data=(X_test, y_test_ohe),
            validation_steps=X_test.shape[0]//BATCH_SIZE,
            verbose=1
           )
)

Epoch 1/3
1562/1562 [==============================] - 1553s 994ms/step - loss: 0.1753 - accuracy: 0.6317 - val_loss: 0.1301 - val_accuracy: 0.7346
Epoch 2/3
1562/1562 [==============================] - 1517s 972ms/step - loss: 0.1176 - accuracy: 0.7636 - val_loss: 0.1145 - val_accuracy: 0.7666
Epoch 3/3
1562/1562 [==============================] - 1530s 979ms/step - loss: 0.1011 - accuracy: 0.7969 - val_loss: 0.1109 - val_accuracy: 0.7766


### <span style='font-family:Georgia'> Residual Network (ResNet)

In [5]:
LR = 2e-5
EPOCHS=3 # 10
INPUT_SHAPE = X_train.shape[1:]

In [6]:
input_tensor=tf.keras.Input(shape=INPUT_SHAPE)

In [7]:
res_base = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
    
resNet = Sequential()
resNet.add(res_base)
resNet.add(tf.keras.layers.Flatten())
resNet.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

In [8]:
resNet.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.RMSprop(learning_rate=LR),
               metrics=['accuracy'])

In [9]:
resNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [10]:
resNet.fit(X_train,
           y_train_ohe,
           batch_size=BATCH_SIZE,
           epochs=EPOCHS,
           shuffle=True,
           verbose=1,
           validation_data=(X_test, y_test_ohe)
          )

Epoch 1/3
1563/1563 [==============================] - 1829s 1s/step - loss: 1.9371 - accuracy: 0.3982 - val_loss: 1.4825 - val_accuracy: 0.5516
Epoch 2/3
1563/1563 [==============================] - 1696s 1s/step - loss: 1.1215 - accuracy: 0.6234 - val_loss: 1.0892 - val_accuracy: 0.6710
Epoch 3/3
1563/1563 [==============================] - 1641s 1s/step - loss: 0.8462 - accuracy: 0.7106 - val_loss: 0.9337 - val_accuracy: 0.7172


## <span style='font-family:Georgia'> Models Evaluation

### <span style='font-family:Georgia'> Residual Network (ResNet)

In [11]:
resNet.evaluate(X_test, y_test_ohe)

313/313 [==============================] - 19s 62ms/step - loss: 0.9337 - accuracy: 0.7172


[0.9337409138679504, 0.717199981212616]

In [33]:
y_pred_resnet = resNet.predict(X_test)

In [13]:
y_pred_resnet_classes = np.argmax(y_pred_resnet, axis=-1)

In [15]:
pd.DataFrame(confusion_matrix(y_test, y_pred_resnet_classes))

,0,1,2,3,4,5,6,7,8,9
0,776,20,38,11,27,4,8,12,75,29
1,27,786,4,10,4,6,9,9,22,123
2,55,5,611,55,109,45,88,19,7,6
3,12,12,57,526,74,168,91,34,10,16
4,15,2,47,46,681,40,95,60,6,8
5,13,2,39,190,53,607,36,49,3,8
6,5,7,40,41,33,32,827,5,4,6
7,14,3,20,38,75,68,9,752,5,16
8,73,42,11,15,12,3,2,4,811,27
9,27,79,5,21,4,11,6,16,36,795


In [17]:
print(classification_report(y_test, y_pred_resnet_classes))

              precision    recall  f1-score   support

           0       0.76      0.78      0.77      1000
           1       0.82      0.79      0.80      1000
           2       0.70      0.61      0.65      1000
           3       0.55      0.53      0.54      1000
           4       0.64      0.68      0.66      1000
           5       0.62      0.61      0.61      1000
           6       0.71      0.83      0.76      1000
           7       0.78      0.75      0.77      1000
           8       0.83      0.81      0.82      1000
           9       0.77      0.80      0.78      1000

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000



### <span style='font-family:Georgia'> VGG16

In [28]:
# TODO
vgg.evaluate(X_test, y_test_ohe)

313/313 [==============================] - 37s 119ms/step - loss: 0.1109 - accuracy: 0.7766


[0.11090455204248428, 0.7766000032424927]

In [34]:
y_pred_vgg16 = vgg.predict(X_test)

In [30]:
y_pred_vgg16_classes = np.argmax(y_pred_vgg16, axis=-1)

In [31]:
pd.DataFrame(confusion_matrix(y_test, y_pred_vgg16_classes))

,0,1,2,3,4,5,6,7,8,9
0,814,23,14,11,11,0,14,8,61,44
1,7,890,0,0,1,1,14,1,9,77
2,54,7,607,33,68,36,156,9,14,16
3,13,14,25,561,46,76,177,25,22,41
4,14,3,34,31,713,14,125,45,18,3
5,8,8,20,163,31,597,103,44,4,22
6,4,5,10,7,5,2,956,1,8,2
7,12,6,14,37,42,30,26,802,2,29
8,24,22,3,3,5,0,5,1,906,31
9,9,48,1,3,0,0,4,2,13,920


In [32]:
print(classification_report(y_test, y_pred_vgg16_classes))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83      1000
           1       0.87      0.89      0.88      1000
           2       0.83      0.61      0.70      1000
           3       0.66      0.56      0.61      1000
           4       0.77      0.71      0.74      1000
           5       0.79      0.60      0.68      1000
           6       0.61      0.96      0.74      1000
           7       0.86      0.80      0.83      1000
           8       0.86      0.91      0.88      1000
           9       0.78      0.92      0.84      1000

    accuracy                           0.78     10000
   macro avg       0.79      0.78      0.77     10000
weighted avg       0.79      0.78      0.77     10000

